In [2]:
#Hasan Rafiq - DNN
#Import required libraries
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import folium

from folium import plugins
print(tf.__version__)

W0320 05:49:29.694199 140377857984384 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


1.13.1


In [3]:
#Check available devices
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5802613291966342372
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 202727095644369061
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 5852279428921471262
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11276946637
locality {
  bus_id: 1
  links {
  }
}
incarnation: 1708933856811126744
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
#Copy training data from drive to local
%%bash
# rm -rf 'training_data'
cp 'drive/My Drive/Colab Notebooks/Kaggle_santander/santander-customer-transaction-prediction.zip' 'file.zip'
unzip -q 'file.zip'

replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)


In [6]:
#Load training data
all_data = pd.read_csv('train.csv',encoding = "cp1252")
all_data.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [7]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Columns: 202 entries, ID_code to var_199
dtypes: float64(200), int64(1), object(1)
memory usage: 308.2+ MB


In [8]:
all_data.columns

Index(['ID_code', 'target', 'var_0', 'var_1', 'var_2', 'var_3', 'var_4',
       'var_5', 'var_6', 'var_7',
       ...
       'var_190', 'var_191', 'var_192', 'var_193', 'var_194', 'var_195',
       'var_196', 'var_197', 'var_198', 'var_199'],
      dtype='object', length=202)

In [9]:
# all_data['location__city_id'] = all_data['location__city_id'].astype(str)
# all_data['location__country_id'] = all_data['location__country_id'].astype(str)
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Columns: 202 entries, ID_code to var_199
dtypes: float64(200), int64(1), object(1)
memory usage: 308.2+ MB


In [10]:
all_data.describe()

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,0.100490,10.679914,-1.627622,10.715192,6.796529,11.078333,-5.065317,5.408949,16.545850,0.284162,...,3.234440,7.438408,1.927839,3.331774,17.993784,-0.142088,2.303335,8.908158,15.870720,-3.326537
std,0.300653,3.040051,4.050044,2.640894,2.043319,1.623150,7.863267,0.866607,3.418076,3.332634,...,4.559922,3.023272,1.478423,3.992030,3.135162,1.429372,5.454369,0.921625,3.010945,10.438015
min,0.000000,0.408400,-15.043400,2.117100,-0.040200,5.074800,-32.562600,2.347300,5.349700,-10.505500,...,-14.093300,-2.691700,-3.814500,-11.783400,8.694400,-5.261000,-14.209600,5.960600,6.299300,-38.852800
25%,0.000000,8.453850,-4.740025,8.722475,5.254075,9.883175,-11.200350,4.767700,13.943800,-2.317800,...,-0.058825,5.157400,0.889775,0.584600,15.629800,-1.170700,-1.946925,8.252800,13.829700,-11.208475
50%,0.000000,10.524750,-1.608050,10.580000,6.825000,11.108250,-4.833150,5.385100,16.456800,0.393700,...,3.203600,7.347750,1.901300,3.396350,17.957950,-0.172700,2.408900,8.888200,15.934050,-2.819550
75%,0.000000,12.758200,1.358625,12.516700,8.324100,12.261125,0.924800,6.003000,19.102900,2.937900,...,6.406200,9.512525,2.949500,6.205800,20.396525,0.829600,6.556725,9.593300,18.064725,4.836800
max,1.000000,20.315000,10.376800,19.353000,13.188300,16.671400,17.251600,8.447700,27.691800,10.151300,...,18.440900,16.716500,8.402400,18.281800,27.928800,4.272900,18.321500,12.000400,26.079100,28.500700


In [0]:
np.random.seed(seed=2)
msk = np.random.rand(len(all_data)) < 0.95
traindf = all_data[msk]
evaldf = all_data[~msk]

#Load submission DF
submissiondf = pd.read_csv('train.csv',encoding='cp1252')

#Files To be used in Tensorflow pipeline dataset API
traindf.to_csv("ml_train.csv", index=False)
evaldf.to_csv("ml_eval.csv", index=False)

In [12]:
print(len(traindf))
print(len(evaldf))

189933
10067


In [13]:
evaldf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10067 entries, 40 to 199929
Columns: 202 entries, ID_code to var_199
dtypes: float64(200), int64(1), object(1)
memory usage: 15.6+ MB


In [14]:
traindf.describe()

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
count,189933.000000,189933.000000,189933.000000,189933.000000,189933.000000,189933.000000,189933.000000,189933.000000,189933.000000,189933.000000,...,189933.000000,189933.000000,189933.000000,189933.000000,189933.000000,189933.000000,189933.000000,189933.000000,189933.000000,189933.000000
mean,0.100635,10.680192,-1.626647,10.716348,6.796357,11.078545,-5.063728,5.408436,16.545660,0.284654,...,3.230549,7.436279,1.927422,3.330179,17.992530,-0.140713,2.303532,8.907708,15.866894,-3.324128
std,0.300846,3.040884,4.051323,2.641985,2.042365,1.623076,7.860397,0.866748,3.417776,3.332404,...,4.558826,3.023244,1.479410,3.991278,3.135536,1.429557,5.452527,0.921176,3.010832,10.438858
min,0.000000,0.408400,-15.043400,2.117100,-0.040200,5.074800,-32.562600,2.347300,5.349700,-10.265300,...,-13.999500,-2.691700,-3.814500,-11.783400,8.694400,-5.261000,-14.209600,5.960600,6.299300,-38.852800
25%,0.000000,8.453700,-4.739600,8.723400,5.255000,9.883100,-11.190800,4.767000,13.943200,-2.317400,...,-0.063700,5.155800,0.888300,0.583400,15.628000,-1.170200,-1.947000,8.252300,13.824300,-11.205800
50%,0.000000,10.525300,-1.609800,10.581200,6.825600,11.108900,-4.827300,5.384400,16.459000,0.394600,...,3.198400,7.344100,1.900500,3.393200,17.956100,-0.172700,2.408200,8.886900,15.930200,-2.818800
75%,0.000000,12.758400,1.360600,12.519400,8.323800,12.262200,0.923700,6.002500,19.099900,2.939400,...,6.403400,9.512000,2.949800,6.207100,20.396500,0.831500,6.555900,9.592700,18.061300,4.838800
max,1.000000,20.315000,10.376800,19.353000,13.188300,16.671400,17.251600,8.447700,27.638900,10.151300,...,18.440900,16.716500,8.402400,18.281800,27.597300,4.272900,18.321500,12.000400,26.079100,28.500700


In [15]:
np.min(traindf["var_0"])

0.4084

In [16]:
#Store scaling information
v_min = {}
v_max = {}

for i in traindf.columns:
    v_min[i] = np.min(traindf[i])
    v_max[i] = np.max(traindf[i])

v_max.pop('ID_code')
v_max.pop('target')
v_min.pop('ID_code')
v_min.pop('target')

#Sanity check
print(v_min['var_0'], v_max['var_0'])

0.4084 20.315


In [0]:
evaldf.describe()

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
count,10067.000000,10067.000000,10067.000000,10067.000000,10067.000000,10067.000000,10067.000000,10067.000000,10067.000000,10067.000000,...,10067.000000,10067.000000,10067.000000,10067.000000,10067.000000,10067.000000,10067.000000,10067.000000,10067.000000,10067.000000
mean,0.097745,10.674670,-1.646011,10.693384,6.799786,11.074332,-5.095306,5.418619,16.549430,0.274871,...,3.307838,7.478591,1.935702,3.361864,18.017442,-0.168034,2.299622,8.916634,15.942917,-3.371979
std,0.296984,3.024434,4.025994,2.620270,2.061329,1.624621,7.917552,0.863939,3.423882,3.337109,...,4.580162,3.023673,1.459728,4.006280,3.128153,1.425702,5.489285,0.930070,3.012316,10.422506
min,0.000000,2.152600,-13.867400,3.318900,0.371100,5.562000,-27.468800,2.643100,6.968500,-10.505500,...,-14.093300,-0.749600,-2.985400,-9.923400,10.434200,-4.644200,-12.635900,6.143200,6.558700,-35.763700
25%,0.000000,8.455350,-4.747150,8.702100,5.235600,9.886100,-11.350000,4.785150,13.950300,-2.332000,...,0.009750,5.204000,0.915750,0.619500,15.665350,-1.185950,-1.941750,8.257550,13.957950,-11.256850
50%,0.000000,10.513800,-1.576100,10.552500,6.809900,11.097300,-4.953600,5.396100,16.421100,0.383200,...,3.295100,7.405700,1.921900,3.458900,17.997700,-0.174000,2.426500,8.906500,15.999300,-2.826900
75%,0.000000,12.750450,1.323450,12.460400,8.333450,12.240150,0.955850,6.016800,19.150750,2.906550,...,6.476450,9.518300,2.940250,6.175700,20.397050,0.795700,6.586500,9.602650,18.122650,4.786200
max,1.000000,19.229200,8.803000,18.247400,12.977300,15.476700,16.128700,8.002100,27.691800,8.926200,...,18.078900,16.392600,7.219100,16.931400,27.928800,3.871800,16.381900,11.827700,26.034600,23.876400


In [0]:
# #Get mean and std of all currencies for scaling
# curr_mean = {}
# curr_std = {}
# mean_curr = traindf.groupby('currency')['average_cost_for_two'].mean()
# std_curr = traindf.groupby('currency')['average_cost_for_two'].std()
# for uq_c in all_data['currency'].unique() and uq_c:
#     print(uq_c)
#     curr_mean[uq_c] = mean_curr[uq_c]
#     curr_std[uq_c] = std_curr[uq_c]

In [0]:
#RETURNS a pre-processing( New columns + Transformations ) function to be used inside pipeline
def create_add_engineered_fn(df):
    #Pass all the above calculated values to be used by main function which will be called in Pipeline
    def fn_add_engineered(features):
#         for k in v_min.keys():
# #             Min max scaling
#             features[k] = tf.divide(tf.cast(tf.subtract(features[k], v_min[k]),tf.float32), 
#                                     tf.cast(tf.subtract( v_max[k], v_min[k]),tf.float32))
# ######             features[k] = ( features[k] - v_min[k] ) / ( v_max[k] - v_min[k] )
        
        return features
    
    return fn_add_engineered

#Generate pre-processing function as per training data
add_engineered = create_add_engineered_fn(traindf.copy())

In [0]:
DEFAULTS = [["train_rec"]]
for i in range(201):
    DEFAULTS.append([0.0])

In [0]:
# Determine CSV, label, and key columns
#Columns in training sheet -> Can have extra columns too
CSV_COLUMNS = traindf.columns
LABEL_COLUMN = 'target'

# Set default values for each CSV column
DEFAULTS = DEFAULTS

def read_dataset(filename, mode, batch_size = 512):
    def _input_fn(v_test=False):     
        # Create list of files that match pattern
        file_list = tf.gfile.Glob(filename)

        if mode == tf.estimator.ModeKeys.TRAIN:
            num_epochs = None # indefinitely
        else:
            num_epochs = 1 # end-of-input after this        
        
        # Create dataset from file list
        #dataset = tf.data.TextLineDataset(file_list).map(decode_csv)
        dataset = tf.contrib.data.make_csv_dataset(file_list,
                                                   batch_size=batch_size,
                                                   column_names=CSV_COLUMNS,
                                                   column_defaults=DEFAULTS,
                                                   label_name=LABEL_COLUMN,
                                                   num_epochs = num_epochs,
                                                   num_parallel_reads=30)

        if mode == tf.estimator.ModeKeys.TRAIN:
            dataset = dataset.shuffle(buffer_size = 10 * batch_size)

        #dataset = dataset.repeat().batch(batch_size)
        batch_features, batch_labels = dataset.make_one_shot_iterator().get_next()
        
        #Begins - Uncomment for testing only -----------------------------------------------------<
        if v_test == True:
            with tf.Session() as sess:
                print(sess.run([add_engineered(batch_features), batch_labels]))
        #End - Uncomment for testing only -----------------------------------------------------<
        return add_engineered(batch_features), batch_labels
    return _input_fn

In [0]:
#Test dataset read + Feat Engg function's - output's CSV + Feature engg columns
eval_file = "\ml_eval.csv"
fn_d = read_dataset(filename = eval_file,
                    mode = tf.estimator.ModeKeys.EVAL,
                    batch_size = 10)

fn_d(v_test=True)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use `tf.data.experimental.make_csv_dataset(...)`.


W0319 04:19:46.994389 139850859079552 deprecation.py:323] From <ipython-input-20-d5c18eb150be>:25: make_csv_dataset (from tensorflow.contrib.data.python.ops.readers) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.data.experimental.make_csv_dataset(...)`.


[OrderedDict([('ID_code', array([b'train_16579', b'train_55977', b'train_121227', b'train_65380',
       b'train_29624', b'train_147333', b'train_99050', b'train_188210',
       b'train_165640', b'train_74616'], dtype=object)), ('var_0', array([ 9.1787, 11.6183,  8.4095,  8.8877,  7.0206,  9.3463, 14.0935,
       10.1029,  9.5439, 14.0428], dtype=float32)), ('var_1', array([-3.8643, -2.2886, -0.0387, -1.4502,  4.563 , -8.7605, -0.3859,
        0.1145, -4.0551, -1.0862], dtype=float32)), ('var_2', array([12.8624,  6.7635, 12.5584,  8.1378,  9.2387, 11.4783,  8.059 ,
       11.5424,  9.3254, 10.827 ], dtype=float32)), ('var_3', array([ 7.7515,  5.0206,  8.4876, 10.044 ,  4.5112,  6.431 ,  9.1395,
        5.7741,  6.3017,  3.7115], dtype=float32)), ('var_4', array([12.299 ,  9.7609, 12.3538, 10.5303,  9.123 , 10.3697, 11.4763,
        8.742 , 13.4661, 12.0907], dtype=float32)), ('var_5', array([  6.3935,  -8.9711, -16.879 , -14.1145, -14.9526, -15.7077,
       -16.9269,   2.2999, -15.3497

(OrderedDict([('ID_code',
               <tf.Tensor 'IteratorGetNext:0' shape=(?,) dtype=string>),
              ('var_0',
               <tf.Tensor 'IteratorGetNext:1' shape=(?,) dtype=float32>),
              ('var_1',
               <tf.Tensor 'IteratorGetNext:2' shape=(?,) dtype=float32>),
              ('var_2',
               <tf.Tensor 'IteratorGetNext:113' shape=(?,) dtype=float32>),
              ('var_3',
               <tf.Tensor 'IteratorGetNext:124' shape=(?,) dtype=float32>),
              ('var_4',
               <tf.Tensor 'IteratorGetNext:135' shape=(?,) dtype=float32>),
              ('var_5',
               <tf.Tensor 'IteratorGetNext:146' shape=(?,) dtype=float32>),
              ('var_6',
               <tf.Tensor 'IteratorGetNext:157' shape=(?,) dtype=float32>),
              ('var_7',
               <tf.Tensor 'IteratorGetNext:168' shape=(?,) dtype=float32>),
              ('var_8',
               <tf.Tensor 'IteratorGetNext:179' shape=(?,) dtype=float32>),
     

In [0]:
# ###Test columns output
# rating_texts = ['Good', 'Average', 'Very Good', 'Excelente', 'Excellent',
#        'Not rated', 'Poor', 'Biasa', 'Harika', 'Çok iyi', 'Bardzo dobrze',
#        'Muy Bueno', 'Bom', 'Buono', 'Terbaik', 'Media', 'İyi', 'Ortalama',
#        'Sangat Baik', 'Velmi dobré', 'Muito Bom', 'Ottimo',
#        'Skvělá volba', 'Priemer', 'Baik', 'Muito bom', 'Veľmi dobré',
#        'Średnio', 'Průměr', 'Bueno', 'Dobrze', 'Promedio', 'Média',
#        'Eccellente', 'Skvělé', 'Malo', 'Vynikajúce',
#        'Belum ada penilaian']

# #dummy data
# training_data = {
#     'rating': tf.constant(['Priemer', 'Baik', 'Muito bom', 'Veľmi dobré',
#        'Średnio', 'Průměr','Belum ada penilaian'])}

# #Create feature columns
# rating_column = tf.feature_column.categorical_column_with_vocabulary_list(
#         'rating', rating_texts)
# rating_column = tf.feature_column.indicator_column(rating_column)
# columns = [
#     rating_column
# ]

# with tf.variable_scope('feature_test'):
#     training_inputs = tf.feature_column.input_layer(training_data, columns)

# var_init = tf.global_variables_initializer()
# table_init = tf.tables_initializer()

# with tf.Session() as sess:
#     sess.run([var_init, table_init])

#     print(sess.run(training_data))
#     print(sess.run(training_inputs))

In [0]:
# Define your feature columns that model will use from (CSV + feat_engg) set to train
def create_feature_cols():  
    
#     deep = [        
        
        
#         latitude,
#         longitude,
#         tf.feature_column.numeric_column('norm_average_cost_for_two'),
#         tf.feature_column.numeric_column('inverse_cuisines'),
#         tf.feature_column.numeric_column('inverse_votes'),
#         tf.feature_column.numeric_column('numb_cuisines'),
#         tf.feature_column.numeric_column('votes_by_cuisines'),
#         tf.feature_column.numeric_column('price_range'),
#         tf.feature_column.numeric_column('inverse_price_range'),
#         tf.feature_column.numeric_column('log_cuisines'),
#         tf.feature_column.numeric_column('log_votes'),
# #         tf.feature_column.numeric_column('price_by_cuisines'),
# #         tf.feature_column.numeric_column('votes_by_price_range'),
# #         tf.feature_column.numeric_column('log_avg_cost'),
#         rating_votes,
#         embed_lat_lon,
#         embed_user_rating_votes,
#         embed_est_type_votes,
# #         embed_col_cuisines
# #         embed_cross_price_range_curr
# #         embed_cost_curr
# #         est_name_embed
#         tf.feature_column.indicator_column(rating_text)
#     ]
    
    deep = []
    wide = []
    for i in range(200):
        col = "var_" + str(i)
        feat_numb = tf.feature_column.numeric_column(col)
        
        #Deep cols
        deep.append(feat_numb)
        
        #Wide cols
        wide.append(tf.feature_column.bucketized_column(feat_numb, boundaries=np.linspace(0, 1, 5).tolist()))
    
    return( wide, deep )

wide, deep = create_feature_cols()
print('Model is using {} features'.format(len(deep + wide)))

Model is using 400 features


In [0]:
# np.linspace(0, 1, 20).tolist() #+ np.linspace(41, 5401, 500).tolist()

In [0]:
for i, c in enumerate(traindf.columns):
    print("Default for {} is {}".format(c, DEFAULTS[i]))

Default for ID_code is ['train_rec']
Default for target is [0.0]
Default for var_0 is [0.0]
Default for var_1 is [0.0]
Default for var_2 is [0.0]
Default for var_3 is [0.0]
Default for var_4 is [0.0]
Default for var_5 is [0.0]
Default for var_6 is [0.0]
Default for var_7 is [0.0]
Default for var_8 is [0.0]
Default for var_9 is [0.0]
Default for var_10 is [0.0]
Default for var_11 is [0.0]
Default for var_12 is [0.0]
Default for var_13 is [0.0]
Default for var_14 is [0.0]
Default for var_15 is [0.0]
Default for var_16 is [0.0]
Default for var_17 is [0.0]
Default for var_18 is [0.0]
Default for var_19 is [0.0]
Default for var_20 is [0.0]
Default for var_21 is [0.0]
Default for var_22 is [0.0]
Default for var_23 is [0.0]
Default for var_24 is [0.0]
Default for var_25 is [0.0]
Default for var_26 is [0.0]
Default for var_27 is [0.0]
Default for var_28 is [0.0]
Default for var_29 is [0.0]
Default for var_30 is [0.0]
Default for var_31 is [0.0]
Default for var_32 is [0.0]
Default for var_33 is

In [0]:
#Create serving function with default values of all the fields
def serving_fn():
#     feature_placeholders = {
#         'average_cost_for_two': tf.placeholder_with_default(-1, None),
#         'currency': tf.placeholder_with_default("-1", None), 
#         'cuisines': tf.placeholder_with_default("", None),
#         'establishment_types__establishment_type__name': tf.placeholder_with_default("-1", None),
#         'has_online_delivery': tf.placeholder_with_default("-1", None),
#         'has_table_booking': tf.placeholder_with_default("-1", None),
#         'is_book_form_web_view': tf.placeholder_with_default("-1", None),
#         'is_delivering_now': tf.placeholder_with_default("-1", None),
#         'is_table_reservation_supported': tf.placeholder_with_default("-1", None),
#         'is_zomato_book_res': tf.placeholder_with_default("-1", None),
#         'price_range': tf.placeholder_with_default(0, None),
#         'mezzo_provider': tf.placeholder_with_default("-1", None),
#         'user_rating__rating_text': tf.placeholder_with_default("-1", None),
#         'location__latitude': tf.placeholder_with_default(0.0, None),
#         'location__longitude': tf.placeholder_with_default(0.0, None),
#         'user_rating__votes': tf.placeholder_with_default(0.0, None),
#         'location__city_id': tf.placeholder_with_default("-1", None),
#         'location__country_id': tf.placeholder_with_default("-1", None),
#         'offers': tf.placeholder_with_default("0", None),
#         'featured_image': tf.placeholder_with_default("no_link", None),
#         'zomato_events': tf.placeholder_with_default("no_event", None),
#         'user_rating__custom_rating_text': tf.placeholder_with_default("no_text", None),
#         'book_url': tf.placeholder_with_default("no_link", None),
#         'numb_cuisines': tf.placeholder_with_default(0.0, None),
#         'opentable_support': tf.placeholder_with_default("1", None),
#         'location__locality': tf.placeholder_with_default("0", None),
#     }
    
    feature_placeholders = {}
    for i in range(200):
        col = "var_" + str(i)
        feature_placeholders[col] = tf.placeholder_with_default(0.0, shape=()) 
    
    features = {
        key: tf.expand_dims(tensor, -1)
        for key, tensor in feature_placeholders.items()
    }
    return tf.estimator.export.ServingInputReceiver(add_engineered(features), feature_placeholders)

In [0]:
def rmse(labels, predictions):
    pred_values = predictions['predictions']
    return {'rmse': tf.metrics.root_mean_squared_error(labels, pred_values)}

In [0]:
# Create estimator train and evaluate function
def train_and_evaluate_dataset(output_dir, num_train_steps, train_file, eval_file):    
##### Create Canned estimator instance
    run_config = tf.estimator.RunConfig(save_checkpoints_secs = 40, 
                                        keep_checkpoint_max = 10)
    
#######DNN Classifier    
#     estimator = tf.estimator.DNNClassifier(
#                                           feature_columns=deep,
# #                                           hidden_units=[256,256,128,128,128,128,64,64],
#                                           hidden_units=[256,512,1024,512,256],
#                                           model_dir=output_dir,
# #                                           batch_norm=True,
#                                           dropout = 0.3,
#                                           activation_fn = tf.nn.relu,
#                                           optimizer= tf.train.AdamOptimizer(learning_rate=0.0001),
# #                                           dnn_optimizer= lambda: tf.train.AdamOptimizer(tf.train.piecewise_constant(
# #                                                             tf.train.get_global_step(), 
# #                                                             [20000], 
# #                                                             [0.01, 0.007])),
#                                           config = run_config)
    
#######Wide&DeepDNN Classifier
    estimator = tf.estimator.DNNLinearCombinedClassifier(
                                          linear_feature_columns=wide,
                                          n_classes = 2,
                                          dnn_feature_columns=deep,
                                          dnn_hidden_units=[256,512,512,256,256],
#                                           batch_norm=True,
#                                           linear_sparse_combiner='mean',
#                                           dnn_dropout = 0.3,
#                                           linear_optimizer=tf.train.FtrlOptimizer(learning_rate=0.1),
                                          linear_optimizer= lambda: tf.train.FtrlOptimizer(
                                                                    learning_rate=0.1,
                                                                    l1_regularization_strength=0.001,
                                                                    l2_regularization_strength=0.001),
                                          dnn_activation_fn = tf.nn.sigmoid,
                                          dnn_optimizer= tf.train.AdamOptimizer(learning_rate=0.001),
#                                           dnn_optimizer= lambda: tf.train.AdamOptimizer(tf.train.piecewise_constant(
#                                                             tf.train.get_global_step(), 
#                                                             [20000], 
#                                                             [0.01, 0.007])),
                                          config = run_config)
    
    train_spec = tf.estimator.TrainSpec(input_fn = read_dataset(
                                                filename = train_file,
                                                mode = tf.estimator.ModeKeys.TRAIN,
                                                batch_size = 128), 
                                      max_steps = num_train_steps)
    exp = tf.estimator.LatestExporter("decision", serving_fn)
    eval_spec = tf.estimator.EvalSpec(input_fn = read_dataset(
                                                filename = eval_file,
                                                mode = tf.estimator.ModeKeys.EVAL,
                                                batch_size = 128), 
                                    steps = None, #Very IMPORTANT COMMENTOUT
                                    exporters = exp,
                                    start_delay_secs = 40, # start evaluating after N seconds, 
                                    throttle_secs = 60)  # evaluate every N seconds
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)
    return estimator

In [0]:
#######BoostedTree Classifier
def gbdt_train_and_evaluate_dataset(output_dir, num_train_steps, train_file, eval_file):
##### Create Canned estimator instance
#     #Distributed training
    devices = ["/device:XLA_GPU:0"]
    strategy = tf.contrib.distribute.OneDeviceStrategy(device='/gpu:0')

    run_config = tf.estimator.RunConfig(save_checkpoints_secs = 40, 
                                        keep_checkpoint_max = 10,)
#                                        train_distribute=strategy, 
#                                         eval_distribute=strategy)

    estimator = tf.estimator.BoostedTreesRegressor(
                                          feature_columns=deep,
                                          n_trees = 100,
                                          max_depth = 6,
                                          model_dir=output_dir,
                                          n_batches_per_layer = 2, ## should be ex: 0.5 * ( total data size / batch size )
                                          learning_rate=0.01,
                                          center_bias= True,
                                          config = run_config)
    
    estimator = tf.contrib.estimator.add_metrics(estimator, rmse)
    
    train_spec = tf.estimator.TrainSpec(input_fn = read_dataset(
                                                filename = train_file,
                                                mode = tf.estimator.ModeKeys.TRAIN,
                                                batch_size = 512), 
                                      max_steps = num_train_steps)
    
    exp = tf.estimator.LatestExporter("decision", serving_fn)
    eval_spec = tf.estimator.EvalSpec(input_fn = read_dataset(
                                                filename = eval_file,
                                                mode = tf.estimator.ModeKeys.EVAL,
                                                batch_size = 512), 
                                    steps = None, #Very IMPORTANT COMMENTOUT
                                    exporters = exp,
                                    start_delay_secs = 40, # start evaluating after N seconds, 
                                    throttle_secs = 60)  # evaluate every N seconds
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)
    return estimator

In [0]:
%%bash
rm -rf "drive/My Drive/Colab Notebooks/Kaggle_santander/model"

# ## Take backup of model
# cp -r "drive/My Drive/Colab Notebooks/Kaggle_santander/model" "drive/My Drive/Colab Notebooks/Kaggle_santander/model_highest_24Feb_95616"

# ## Retrieve from backup
# cp -r "drive/My Drive/Colab Notebooks/Kaggle_santander/model_highest_9584" "drive/My Drive/Colab Notebooks/Kaggle_santander/model"

In [0]:
#tf.logging.set_verbosity(tf.logging.WARN)
train_file = "\ml_train.csv"
eval_file = "\ml_eval.csv"
outdir =  r"drive/My Drive/Colab Notebooks/Kaggle_santander/model"

# estimator = train_and_evaluate_dataset(outdir, 50000, train_file, eval_file)
estimator = gbdt_train_and_evaluate_dataset(outdir, None, train_file, eval_file)

INFO:tensorflow:Using config: {'_model_dir': 'drive/My Drive/Colab Notebooks/Kaggle_santander/model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 40, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 10, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f310f000390>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0319 04:23:46.075295 139850859079552 estimator.py:201] Using config: {'_model_dir': 'drive/My Drive/Colab Notebooks/Kaggle_santander/model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 40, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 10, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f310f000390>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': 'drive/My Drive/Colab Notebooks/Kaggle_santander/model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 40, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 10, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f310f000470>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0319 04:23:46.091936 139850859079552 estimator.py:201] Using config: {'_model_dir': 'drive/My Drive/Colab Notebooks/Kaggle_santander/model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 40, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 10, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f310f000470>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Not using Distribute Coordinator.


...

W0319 04:24:08.236183 139850859079552 meta_graph.py:445] Issue encountered when serializing resources.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'


INFO:tensorflow:Saving checkpoints for 0 into drive/My Drive/Colab Notebooks/Kaggle_santander/model/model.ckpt.


I0319 04:24:08.568881 139850859079552 basic_session_run_hooks.py:594] Saving checkpoints for 0 into drive/My Drive/Colab Notebooks/Kaggle_santander/model/model.ckpt.


Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'


W0319 04:24:08.938229 139850859079552 meta_graph.py:445] Issue encountered when serializing resources.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'


In [0]:
# #Evaluate full file
# estimator.evaluate(input_fn = read_dataset(
#                                         filename = eval_file,
#                                         mode = tf.estimator.ModeKeys.EVAL,
#                                         batch_size = 1),steps=1)

In [0]:
# #Run prediction
# def read_pred_dataset(filename, mode, batch_size = 512):
#     def _input_fn(v_test=False):     
#         # Create list of files that match pattern
#         file_list = tf.gfile.Glob(filename)

#         # Create dataset from file list
#         #dataset = tf.data.TextLineDataset(file_list).map(decode_csv)
#         dataset = tf.contrib.data.make_csv_dataset(file_list,
#                                                    batch_size=batch_size,
#                                                    column_names=CSV_COLUMNS,
#                                                    column_defaults=DEFAULTS,
#                                                    label_name=LABEL_COLUMN,
#                                                    shuffle=False)
#         #dataset = dataset.repeat().batch(batch_size)
#         batch_features, batch_labels = dataset.make_one_shot_iterator().get_next()
        
#         #Begins - Uncomment for testing only -----------------------------------------------------<
#         if v_test == True:
#             with tf.Session() as sess:
#                 print(sess.run([add_engineered(batch_features), batch_labels]))
#         #End - Uncomment for testing only -----------------------------------------------------<
#         return add_engineered(batch_features)
#     return _input_fn

# pred_file = os.getcwd() + "\pred_test.csv"
# pr = estimator.predict(input_fn = read_pred_dataset(
#                                         filename = pred_file,
#                                         mode = tf.estimator.ModeKeys.EVAL,
#                                         batch_size = 1))
# for i in pr:
#     print(i)

In [0]:
#Get evaluation RMSE on eval file
from tensorflow.contrib import predictor

export_dir = os.getcwd() + "\model2"
# export_dir = r"C:\Users\hrafiq\AppData\Local\Temp\tmpxq2m_h7l\export\decision\1545063735" 
predict_fn = predictor.from_saved_model(export_dir,signature_def_key='predict')
#predictions = predict_fn({"text":["hello how are you"]})  #Dummy request

# test_file = os.getcwd() + "\Zomato Rating Prediction\\Eatery_TestData.csv"
test_file = os.getcwd() + "\ml_eval.csv"
test_pd = pd.read_csv(test_file, keep_default_na=False)
print(test_pd.columns)

row={}
out=[]

sum = 0
for k, d in test_pd.iterrows():
    row = {}
    row['ID_code'] = d['ID_code']
    row['actual_target'] = d['target']
    
#   payload
    payload = {
#             'average_cost_for_two': [d['average_cost_for_two']],
#             'currency': [str(d['currency'])],
#             'cuisines': [str(d['cuisines'])],
#             'establishment_types__establishment_type__name': [str(d['establishment_types__establishment_type__name'])],
#             'has_online_delivery': [str(d['has_online_delivery'])],
#             'has_table_booking': [str(d['has_table_booking'])],
#             'is_book_form_web_view': [str(d['is_book_form_web_view'])],
#             'is_delivering_now': [str(d['is_delivering_now'])],
#             'is_table_reservation_supported': [str(d['is_table_reservation_supported'])],
#             'is_zomato_book_res': [str(d['is_zomato_book_res'])],
#             'price_range': [str(d['price_range'])],
#             'mezzo_provider': [str(d['mezzo_provider'])],
#             'user_rating__rating_text': [str(d['user_rating__rating_text'])],
#             'location__latitude': [d['location__latitude']],
#             'location__longitude': [d['location__longitude']],
#             'user_rating__votes': [d['user_rating__votes']],
#             'location__city_id': [str(d['location__city_id'])],
#             'location__country_id': [str(d['location__country_id'])],
#             'offers': [str(d['offers'])], #tf.placeholder(tf.string, [None]),
#             'featured_image': [str(d['featured_image'])], #tf.placeholder(tf.string, [None]),
#             'zomato_events': [str(d['zomato_events'])], #tf.placeholder(tf.string, [None]),
#             'user_rating__custom_rating_text': [str(d['user_rating__custom_rating_text'])], #tf.placeholder(tf.string, [None]),
#             'book_url': [str(d['book_url'])], #tf.placeholder(tf.string, [None])
#             'numb_cuisines': [d['numb_cuisines']],
#             'opentable_support': [str(d['opentable_support'])],
#             'location__locality': [str(d['location__locality'])]
        }
    
    payload = {}
    for i in range(200):
        col = "var_" + str(i)
        payload[col] = d[col]
    
    print(predict_fn(payload))
    prob_to_close = predict_fn(payload)['probabilities'][0][1]
    row['predicted_target'] = prob_to_close
    out.append(row)
    break
    
    #RMSE
#     sum = sum + (row['predicted_rating'] - row['actual_rating'] )**2
    
out_csv = pd.DataFrame(out)
# print(sum)
# print(len(out_csv))
out_csv.to_csv("eval_out_test.csv", index=False,columns=['ID_code','actual_target','predicted_target'])
# print("RMSE is .. ", (sum / len(out_csv))**0.5)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from C:\Users\hrafiq\Documents\HRafiq\hrafiq\Machine_Deep_learning\Deep learning\Kaggle_Santander\model2\variables\variables
Index(['ID_code', 'target', 'var_0', 'var_1', 'var_2', 'var_3', 'var_4',
       'var_5', 'var_6', 'var_7',
       ...
       'var_190', 'var_191', 'var_192', 'var_193', 'var_194', 'var_195',
       'var_196', 'var_197', 'var_198', 'var_199'],
      dtype='object', length=202)
{'logits': array([[-2.9832392]], dtype=float32), 'classes': array([[b'0']], dtype=object), 'probabilities': array([[0.9518112 , 0.04818884]], dtype=float32), 'class_ids': array([[0]], dtype=int64), 'logistic': array([[0.04818884]], dtype=float32)}


In [0]:
#Predict with disk saved model
from tensorflow.contrib import predictor

export_dir = os.getcwd() + "\model1"
predict_fn = predictor.from_saved_model(export_dir,signature_def_key='predict')
#predictions = predict_fn({"text":["hello how are you"]})  #Dummy request

test_file = os.getcwd() + r"\santander-customer-transaction-prediction\test.csv"
test_pd = pd.read_csv(test_file, keep_default_na=False,encoding = "cp1252")

row={}
out=[]

for k, d in test_pd.iterrows():
    row = {}
    row['ID_code'] = d['ID_code']
    
#   payload
    payload = {}
    for i in range(200):
        col = "var_" + str(i)
        payload[col] = d[col]
    
    prob_to_close = predict_fn(payload)['probabilities'][0][1]
    row['target'] = prob_to_close
    out.append(row)
    
out_csv = pd.DataFrame(out)
out_csv.to_csv("submission.csv", index=False,columns=['ID_code','target'])

INFO:tensorflow:Restoring parameters from C:\Users\hrafiq\Documents\HRafiq\hrafiq\Machine_Deep_learning\Deep learning\Kaggle_Santander\model1\variables\variables


In [0]:
#Predict with disk saved model
from tensorflow.contrib import predictor

export_dir = os.getcwd() + "\model7"
predict_fn = predictor.from_saved_model(export_dir,signature_def_key='predict')
# predict_fn = predictor.from_estimator(estimator, serving_input_receiver_fn=serving_fn)

load = {'location__city_id': ['86'], 'mezzo_provider': ['OTHER'], 'user_rating__votes': [918], 'currency': ['LBP'], 
        'is_zomato_book_res': ['0'], 'user_rating__rating_text': ['Very Good'], 'zomato_events': [''], 
        'user_rating__custom_rating_text': [''], 'has_online_delivery': ['0'], 
        'featured_image': ['https://b.zmtcdn.com/data/pictures/9/18411489/eb48166eed52472417a0829c2d97f9f7.jpg'], 
        'offers': ['[]'], 'book_url': [''], 'is_book_form_web_view': ['0'], 'has_table_booking': ['0'], 
        'cuisines': ['American, Salad, Burger, Pizza, Sandwich'], 'is_table_reservation_supported': ['0'], 
        'location__latitude': [34.09350744], 'is_delivering_now': ['0'], 'average_cost_for_two': [35000], 
        'location__country_id': ['112'], 'price_range': ['2'], 
        'establishment_types__establishment_type__name': ['Quick Bites'], 'location__longitude': [35.65133583]}

predict_fn(load)

# **LGBM model**

In [0]:
# #LIGHTGBM: GPU build

# !git clone --recursive https://github.com/Microsoft/LightGBM ; cd LightGBM
# import os
# os.chdir('LightGBM')
# !mkdir build
# os.chdir('build')
# !apt-get install -qq cmake
# !cmake -DUSE_GPU=1 ..
# !make -j4

Cloning into 'LightGBM'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 12721 (delta 15), reused 17 (delta 8), pack-reused 12684
Receiving objects: 100% (12721/12721), 9.29 MiB | 11.38 MiB/s, done.
Resolving deltas: 100% (8994/8994), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'compute'
Cloning into '/content/LightGBM/compute'...
remote: Enumerating objects: 14, done.        
remote: Counting objects: 100% (14/14), done.        
remote: Compressing objects: 100% (11/11), done.        
remote: Total 21670 (delta 3), reused 8 (delta 2), pack-reused 21656        
Receiving objects: 100% (21670/21670), 8.51 MiB | 10.23 MiB/s, done.
Resolving deltas: 100% (17523/17523), done.
Submodule path 'compute': checked out '509ebe4a9282eec8a92c65ce3bbc1925f1fdbe07'
-- The C compiler identification is GNU 7.3.0
-- The CXX compiler identification i

In [0]:
# # !pip uninstall lightgbm
# !pip install lightgbm --install-option=--gpu

/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:207: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)


In [0]:
import lightgbm as lgb

In [18]:
import math
math.sin(math.pi / 2)

1.0

In [0]:
#Function to write feature engg
import math

def do_feature_engg(features):
  excl_list = ['ID_code','target']
  features_scaled = features.copy()
  
  #Original cols
  orig_cols = features.columns
  
  #Do FT engg
  #Min
  features['v_min'] = features.min(axis=1)

  #Max
  features['v_max'] = features.max(axis=1)

  #Average
  features['v_mean'] = features.mean(axis=1)
  
  #Calculated columns using original DF
  for c in orig_cols:
    if not c in excl_list:
      #Scaling
      features_scaled[c] = ( features[c] - v_min[c] ) / ( v_max[c] - v_min[c] )
      
      #Power
      features['v_squared' + c] = features[c] ** ( 1 / 3 )
      
      #Sin
      features['v_sin' + c] = np.sin(features_scaled[c] * (math.pi / 2))
      
      #Cos
      features['v_cos' + c] = np.cos(features_scaled[c] * (math.pi / 2))
    
  return(features)

In [0]:
#Do feature engineering
traindf_ft = do_feature_engg(traindf.copy())
evaldf_ft = do_feature_engg(evaldf.copy())

In [38]:
traindf_ft.describe()

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,v_cosvar_196,v_squaredvar_197,v_sinvar_197,v_cosvar_197,v_squaredvar_198,v_sinvar_198,v_cosvar_198,v_squaredvar_199,v_sinvar_199,v_cosvar_199
count,189933.000000,189933.000000,189933.000000,189933.000000,189933.000000,189933.000000,189933.000000,189933.000000,189933.000000,189933.000000,...,1.899330e+05,189933.000000,189933.000000,1.899330e+05,189933.000000,189933.000000,1.899330e+05,77967.000000,189933.000000,1.899330e+05
mean,0.100635,10.680192,-1.626647,10.716348,6.796357,11.078545,-5.063728,5.408436,16.545660,0.284654,...,6.746321e-01,2.070456,0.673978,6.998973e-01,2.502177,0.669616,7.042526e-01,1.778679,0.715644,6.558299e-01
std,0.300846,3.040884,4.051323,2.641985,2.042365,1.623076,7.860397,0.866748,3.417776,3.332404,...,1.847059e-01,0.071970,0.170682,1.636023e-01,0.164600,0.173861,1.594229e-01,0.495835,0.165983,1.737537e-01
min,0.000000,0.408400,-15.043400,2.117100,-0.040200,5.074800,-32.562600,2.347300,5.349700,-10.265300,...,6.123234e-17,1.813134,0.000000,6.123234e-17,1.846846,0.000000,6.123234e-17,0.000000,0.000000,6.123234e-17
25%,0.000000,8.453700,-4.739600,8.723400,5.255000,9.883100,-11.190800,4.767000,13.943200,-2.317400,...,5.380430e-01,2.020808,0.561347,5.860544e-01,2.400017,0.562653,5.945669e-01,1.454359,0.601018,5.242508e-01
50%,0.000000,10.525300,-1.609800,10.581200,6.825600,11.108900,-4.827300,5.384400,16.459000,0.394600,...,6.949781e-01,2.071334,0.689686,7.241086e-01,2.516172,0.692414,7.215002e-01,1.844537,0.744892,6.671849e-01
75%,0.000000,12.758400,1.360600,12.519400,8.323800,12.262200,0.923700,6.002500,19.099900,2.939400,...,8.297639e-01,2.124778,0.810272,8.275808e-01,2.623713,0.804046,8.266931e-01,2.163908,0.851564,7.992354e-01
max,1.000000,20.315000,10.376800,19.353000,13.188300,16.671400,17.251600,8.447700,27.638900,10.151300,...,1.000000e+00,2.289454,1.000000,1.000000e+00,2.965497,1.000000,1.000000e+00,3.054582,1.000000,1.000000e+00


In [0]:
#Create train validation sets
x_train = traindf_ft.drop(['target','ID_code'], axis=1)
y_train = traindf_ft['target']

#Create train validation sets
x_eval = evaldf_ft.drop(['target','ID_code'], axis=1)
y_eval = evaldf_ft['target']

In [0]:
#Create LGBM instance
train_data = lgb.Dataset(x_train, label=y_train)
val_data = lgb.Dataset(x_eval, label=y_eval)

params = {}
params['learning_rate'] = 0.008
params['boosting_type'] = 'gbdt'
params['bagging_freq'] = 10
params['bagging_fraction'] = 0.25    #Helps increase training speed
params['objective'] = 'binary'
params['metric'] = 'auc'
params['sub_feature'] = 0.8
params['num_leaves'] = 20
params['feature_fraction'] = 0.25    #Lower feature fraction helps fight overfit
params['min_data'] = 80
params['max_depth'] = -1
params['lambda_l1'] = 0.0015
params['lambda_l2'] = 0.0015
params['verbosity'] = 1
params['num_threads'] = 8
params['device_type'] = 'cpu'
params['tree_learner'] = 'feature' #serial
# params['categorical_feature '] = 'name:var_68'

In [41]:
#Train
early_stopping_rounds = 1500

clf = lgb.train(params, train_data, 50000,
                early_stopping_rounds = early_stopping_rounds , valid_sets = [train_data, val_data], verbose_eval=500)

Training until validation scores don't improve for 1500 rounds.
[500]	training's auc: 0.869701	valid_1's auc: 0.83223
[1000]	training's auc: 0.898985	valid_1's auc: 0.857156
[1500]	training's auc: 0.914112	valid_1's auc: 0.86814
[2000]	training's auc: 0.923429	valid_1's auc: 0.874674
[2500]	training's auc: 0.930118	valid_1's auc: 0.877544
[3000]	training's auc: 0.93563	valid_1's auc: 0.88013
[3500]	training's auc: 0.940099	valid_1's auc: 0.881838
[4000]	training's auc: 0.944192	valid_1's auc: 0.882373
[4500]	training's auc: 0.94801	valid_1's auc: 0.883158
[5000]	training's auc: 0.951522	valid_1's auc: 0.883332
[5500]	training's auc: 0.954956	valid_1's auc: 0.88374
[6000]	training's auc: 0.958162	valid_1's auc: 0.883542
[6500]	training's auc: 0.9613	valid_1's auc: 0.883487
[7000]	training's auc: 0.964407	valid_1's auc: 0.883432
Early stopping, best iteration is:
[5503]	training's auc: 0.954971	valid_1's auc: 0.883754


In [0]:
# os.chdir('../..')

# **CATBOOST**

In [42]:
# !pip install catboost

    100% |████████████████████████████████| 60.0MB 408kB/s 


In [53]:
#CATBOOST model
from catboost import CatBoostClassifier

early_stopping_rounds=1500

model = CatBoostClassifier(
    iterations=50000,
    learning_rate=0.01,
    task_type='GPU',
    loss_function = 'Logloss',
    eval_metric='AUC',
    metric_period=500,
#     rsm=0.25,           #Percentage of features at each level
    depth=8,
    od_type='Iter',       #Over-Fitting detection is off(early stopping passed)
    early_stopping_rounds = early_stopping_rounds,
    l2_leaf_reg=6,
    data_partition='FeatureParallel'
  )

model.fit(
      x_train, y_train,
      eval_set=(x_eval, y_eval),
      verbose=500
  );  

0:	learn: 0.6566133	test: 0.6387807	best: 0.6387807 (0)	total: 113ms	remaining: 1h 34m 17s
500:	learn: 0.8550602	test: 0.8228657	best: 0.8228657 (500)	total: 33.3s	remaining: 54m 49s
1000:	learn: 0.8855889	test: 0.8496625	best: 0.8496625 (1000)	total: 1m 7s	remaining: 55m 7s
1500:	learn: 0.9025026	test: 0.8620073	best: 0.8620073 (1500)	total: 1m 42s	remaining: 55m 19s
2000:	learn: 0.9137387	test: 0.8683592	best: 0.8683592 (2000)	total: 2m 18s	remaining: 55m 22s
2500:	learn: 0.9221444	test: 0.8723982	best: 0.8724042 (2499)	total: 2m 53s	remaining: 55m 2s
3000:	learn: 0.9288159	test: 0.8754039	best: 0.8754039 (3000)	total: 3m 28s	remaining: 54m 31s
3500:	learn: 0.9345181	test: 0.8772707	best: 0.8772707 (3500)	total: 4m 4s	remaining: 54m 8s
4000:	learn: 0.9394697	test: 0.8786486	best: 0.8786486 (4000)	total: 4m 39s	remaining: 53m 37s
4500:	learn: 0.9438404	test: 0.8796407	best: 0.8796407 (4500)	total: 5m 14s	remaining: 52m 59s
5000:	learn: 0.9477493	test: 0.8804564	best: 0.8804564 (5000)	

In [0]:
#Run on submission file
test_file = r"test.csv"
test_df = pd.read_csv(test_file, keep_default_na=False,encoding = "cp1252")

test_pd = do_feature_engg(test_df.copy())

In [0]:
#Prepare submission input
x_subm_id = test_pd.ID_code.values
x_subm = test_pd.drop(['ID_code'], axis=1)

#Predict
prob = clf.predict(x_subm)

#save submissions
sub = pd.DataFrame({"ID_code": x_subm_id})
sub["target"] = prob
sub.to_csv("submission.csv", index=False)